#EV Charger Route Planning using Open Charge Map and Open Street Map
Open Charge Map (https://openchargemap.org/site) is an open data source that provides EV charger location and characteristics data.
Open Route Service (https://openrouteservice.org/) is a open data source that provides map related services including navigation.

This Python Notebook performs 3 functions
1. Using Open Charge Map a list of EV chargers with characteristics (including location data) is extracted and saved as a dataframe (df_ev_chargers)
2. Based on user defined parameters Open Charger Map is used to find and define a route from Origin to Destination through the EV charging network.  The Algorithin used executes the following steps
    - (a) Set the start as the Origin.  Calculate (Open Route Service Matrix Function) the driving distance from the origin to each EV charger.  Reduce the list to only EV chargers that are within current range of the vehicle.
    - Calculate (Open Route Service Matrix Function) the driving distance from the final destination to each EV charger from step (a)
    - Select the EV charger that is closest to the destination.
    - Set the new origin as the EV charger and repeat untill you can reach the final destination without any more EV charging stops

3. Once a route has been determines then the route is plotted and presented using a Map Object.


# API KEYS
API Keys are required to make calles to Open Charge Map and Open Route Service.
Please update the below to include your own keys.  There are limits on the number of calls that can be made to both Open Charge Map and Open Route Service within periods.  The remaining code will fail once the key limit has been reached and the user will need to wait 24 hours before being able to use the key again.

In [1]:
api_key_openchargemap = 'fbc86873-6b79-4f17-bdf0-e97d91fcb9e3'
api_key_openrouteservice = '5b3ce3597851110001cf6248c7ae45297d34493ea117f6002af2b8f1' #Personal


# Define Variables and Constants
- const_maxstops: The route planning algorithm can get lost in some scenarios.  Therefore a limit of 20 stops has been implemented.  Testing shows that for the default setup on the current EV charger network it takes 14 stops to navigate from Sydney to Perth.  Therefore is the system reaches 20 stops it is assumed that the algorithm has failed to find a route and will exit the loop
- user_maxrange: This is the range of the vehicle when charged to 100%
- user_startingpercent: This is the current charge percent of the vehicle at the start of the journey.
- user_safetymargin: This is the amount of charge the user would like remaining when they reach a charging station.  Think of this like the petrol light.  The algorithm will limit the driving distance such that the vehicle charge percent will not go below the safety margin
- user_maxchargepercent: When stopping and charging at charging stations the rate of charge is dependent on the current charge.  Charging vehicles from 80% to 100% is typically much slower than the rate of charge from 10% to 80%.  Therefore when travelling it is unlikely you will charge to 100% unless stopped overnight.  User can define what percent they would like to charge to with a default industry standard of 80%
-op_startingrange: The calculated starting range of the vehicle in km.  This is determined based on maxrange, startingpercent and safetymargin
-op_normalrange: The calculated range after each stop at a charging station assuming the user chargers to the maxchargepercent

In [2]:
const_maxstops = 20

user_maxrange = 400.0
user_startingpercent = 0.95
user_safetymargin = 0.05
user_maxchargepercent = 0.8

op_startingrange = (user_maxrange * user_startingpercent) - (user_maxrange * user_safetymargin)
op_normalrange = user_maxrange * user_maxchargepercent

print("Operating Starting Range:", op_startingrange)
print("Operating Normal Range:", op_normalrange)

Operating Starting Range: 360.0
Operating Normal Range: 320.0


#PIP Installs
-requests (For HTTP Requests to Open Charge Map)
-openrouteservice (Open Route Service has a Python Library available)
-polyline (For plotting on a Map)

In [3]:
pip install requests

In [4]:
pip install openrouteservice

In [5]:
pip install polyline

# Location of top 50 (by population) cities and towns in Australia
This list is used in the UI to allow users to select navigation from origin to destination across these 50 cities

In [6]:
import pandas as pd
import requests

australian_cities_location = {
    "Sydney": [151.2093, -33.8688],
    "Melbourne": [144.9631, -37.8136],
    "Brisbane": [153.0251, -27.4698],
    "Perth": [115.8605, -31.9505],
    "Adelaide": [138.6007, -34.9285],
    "Gold Coast": [153.4000, -28.0167],
    "Canberra": [149.1300, -35.2809],
    "Newcastle": [151.7789, -32.9267],
    "Central Coast": [151.2333, -33.2833],
    "Sunshine Coast": [153.0667, -26.6500],
    "Wollongong": [150.8931, -34.4278],
    "Hobart": [147.3250, -42.8821],
    "Geelong": [144.3500, -38.1500],
    "Townsville": [146.8139, -19.2580],
    "Cairns": [145.7700, -16.9186],
    "Darwin": [130.8456, -12.4634],
    "Toowoomba": [151.9555, -27.5614],
    "Ballarat": [143.8503, -37.5622],
    "Bendigo": [144.2811, -36.7570],
    "Albury-Wodonga": [146.9278, -36.0737],
    "Mackay": [149.1860, -21.1412],
    "Rockhampton": [150.5044, -23.3774],
    "Launceston": [147.1543, -41.4381],
    "Bunbury": [115.6383, -33.3256],
    "Hervey Bay": [152.8400, -25.2888],
    "Maitland": [151.5500, -32.7333],
    "Wagga Wagga": [147.3636, -35.1150],
    "Coffs Harbour": [153.1250, -30.2963],
    "Mildura": [142.1625, -34.2083],
    "Shepparton": [145.3889, -36.3833],
    "Gladstone": [151.2583, -23.8478],
    "Tamworth": [150.9167, -31.0833],
    "Port Macquarie": [152.9185, -31.4333],
    "Orange": [149.1000, -33.2833],
    "Dubbo": [148.6167, -32.2500],
    "Geraldton": [114.6000, -28.7667],
    "Nowra": [150.6000, -34.8833],
    "Bathurst": [149.5765, -33.4193],
    "Warrnambool": [142.4794, -38.3817],
    "Lismore": [153.2744, -28.8135],
    "Albany": [117.8814, -35.0231],
    "Kalgoorlie-Boulder": [121.4667, -30.7500],
    "Devonport": [146.3419, -41.1770],
    "Mount Gambier": [140.7800, -37.8284],
    "Burnie": [145.9167, -41.0500],
    "Whyalla": [137.5833, -33.0333]
}

# Get All EV Chargers
This function uses Open Charge Map to search for and return a dataframe with the extended attributes of each EV charger in Australia

In [7]:
def get_all_ev_chargers(api_key, country_code="AU", distance=None, max_results=None):
    # Open Charge Map API endpoint for fetching EV charger data
    api_url = "https://api.openchargemap.io/v3/poi/"

    # Parameters for the API request
    params = {
        "countrycode": country_code,  # Specify the country code for Australia
        "compact": False,  # Get full details
        "verbose": True,  # Get extended data
        "key": api_key  # Your Open Charge Map API key
    }

    if distance is not None:
        params["distance"] = distance  # Search radius in kilometers

    if max_results is not None:
        params["maxresults"] = max_results  # Maximum number of results to return

    try:
        # Send GET request to Open Charge Map API
        response = requests.get(api_url, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors
        ev_chargers_data = response.json()
        return ev_chargers_data
    except requests.RequestException as e:
        print("Error fetching EV charger data:", e)
        return None

def create_dataframe(ev_chargers_data):
    # Create DataFrame from EV chargers data
    df = pd.json_normalize(ev_chargers_data)
    return df

In [8]:
ev_chargers = get_all_ev_chargers(api_key_openchargemap, max_results=2000)
if ev_chargers:
    print("Found", len(ev_chargers), "EV chargers in Australia.")

    # Convert data to DataFrame
    df_ev_chargers = create_dataframe(ev_chargers)
    df_connections = df_ev_chargers[['ID', 'Connections']]
    df_ev_chargers_location = df_ev_chargers[['AddressInfo.Longitude','AddressInfo.Latitude']]
    ls_ev_chargers_location = df_ev_chargers_location.values.tolist()

else:
    print("Failed to fetch EV charger data.")

df_ev_chargers

Found 1101 EV chargers in Australia.


,UserComments,PercentageSimilarity,MediaItems,IsRecentlyVerified,DateLastVerified,ID,UUID,ParentChargePointID,DataProviderID,DataProvidersReference,...,AddressInfo.Longitude,AddressInfo.ContactTelephone1,AddressInfo.ContactTelephone2,AddressInfo.ContactEmail,AddressInfo.AccessComments,AddressInfo.RelatedURL,AddressInfo.Distance,AddressInfo.DistanceUnit,OperatorInfo,UsageType
0,None,None,None,True,2024-05-09T00:19:00Z,298483,BFACB6EE-C799-4D7B-942D-E143521CDF4B,None,1,None,...,119.827157,None,None,None,None,None,None,0,NaN,NaN
1,None,None,None,True,2024-05-10T08:24:00Z,298208,255A6AF1-2354-4EBD-88D8-3A9369CF7055,None,1,None,...,151.213645,None,None,None,None,None,None,0,NaN,NaN
2,"[{'ID': 29709, 'ChargePointID': 298207, 'Comme...",None,"[{'ID': 42209, 'ChargePointID': 298207, 'ItemU...",True,2024-05-10T08:24:00Z,298207,C0201F83-3576-4EB2-AD4E-79CEA0D152BD,None,1,None,...,151.213645,None,None,None,None,None,None,0,NaN,NaN
3,None,None,None,True,2024-05-10T08:24:00Z,298092,4F969ED5-E93C-4AA2-8D58-37BCD041D187,None,1,None,...,139.986310,None,None,None,None,None,None,0,NaN,NaN
4,None,None,None,True,2024-05-10T08:24:00Z,298091,408D0C1F-E0E0-4C60-B34F-9215323EDE6B,None,1,None,...,139.996613,+61 1301518038,None,None,None,None,None,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1096,"[{'ID': 364, 'ChargePointID': 9892, 'CommentTy...",None,None,False,2017-11-04T14:03:00Z,9892,E12E25BD-476E-4106-9C41-9170B63AAF68,None,15,17118,...,151.018619,(02) 9653-9385,None,None,None,http://carstations.com/17118,None,0,NaN,NaN
1097,None,None,None,False,2023-10-23T06:42:00Z,8745,07FAB3FC-A1AD-4A88-93D1-866F298F350C,None,1,None,...,115.840552,0422542326,None,None,None,None,None,0,NaN,NaN
1098,None,None,None,False,2017-11-04T14:03:00Z,8740,D4EE035B-CAFC-45DA-B815-FAC76A814957,None,15,271,...,149.087110,,None,None,None,http://carstations.com/271,None,0,NaN,NaN
1099,None,None,None,False,2017-11-04T14:03:00Z,8737,4068C23F-BE78-42B6-AD76-83F8D20584AB,None,15,278,...,151.191530,0406868359,None,None,None,http://carstations.com/278,None,0,NaN,NaN


# Get Distance and Duration
This function runs a Matrix (https://openrouteservice.org/services/) to return the Time (duration) and Distance from an origin to a list of destinations.  The result is returned as a dataframe that has the Origin, each destination and the duration (seconds) and distance (km) from origin to each destination.  When the destinations are passed as the list of EV charger locations the function returns the duration and distance from origin to each EV charger.

In [9]:
import openrouteservice
import json

def get_distance_duration(origin, destinations, api_key):
    # Set up OpenRouteService client
    client = openrouteservice.Client(key=api_key)

    # Define the coordinates for origin and destinations
    coordinates = [origin] + destinations

    # Get the distance matrix
    matrix = client.distance_matrix(coordinates, sources=[0], profile='driving-car', units='km', metrics = ['distance', 'duration'])
    # Extract distances and durations
    results = []
    for dest_index, destination in enumerate(destinations):
        duration = matrix['durations'][0][dest_index + 1]  # Duration from origin to destination
        distance = matrix['distances'][0][dest_index + 1]  # Duration from origin to destination
        results.append({
            'origin': origin,
            'ev_charger': destination,
            'duration': duration,
            'distance': distance
        })

    return results

# Find Optimal EV Charger
The algorithm will return the EV charger that is within range of the Origin and is the closest to the final destination

1. Get the distance/duration from the journey Origin to each EV charger
2. Remove EV chargers from the list that are outside the current driving range
3. Get the distance/duration from the journey destination to each EV charger from step 2.
4. Sort the list based on distance to destination asc
5. The first item in the list is the EV charger that is within range of the origin and closest to the destination




In [10]:
def find_optimal_ev_charger(origin, destinations, list_ev_chargers, driving_range):
  df_distancefromorigin_all = get_distance_duration(origin, list_ev_chargers, api_key_openrouteservice)
  df_distancefromorigin_all = pd.DataFrame(df_distancefromorigin_all)
  df_distancefromorigin_all = df_distancefromorigin_all.assign(ID=pd.Series(df_ev_chargers['ID']).values) #Add the charger ID back to the data
  df_distancefromorigin_all['destination'] = df_distancefromorigin_all.apply(lambda x: destinations, axis = 1) #Add the origin to the data

  df_distancefromorigin = df_distancefromorigin_all.dropna()
  df_distancefromorigin = df_distancefromorigin.drop(df_distancefromorigin[df_distancefromorigin.distance > driving_range].index)
  df_distancefromorigin = df_distancefromorigin.reset_index(drop=True)

  df_distancefromdestination_all = get_distance_duration(destinations, df_distancefromorigin['ev_charger'].values.tolist(), api_key_openrouteservice)
  df_distancefromdestination_all = pd.DataFrame(df_distancefromdestination_all)

  df_distancefromorigin['distance_destination'] = df_distancefromdestination_all['distance']
  df_distancefromorigin['duration_destination'] = df_distancefromdestination_all['duration']

  df_distancefromorigin = df_distancefromorigin.sort_values(by=['distance_destination'], ascending=True)
  df_distancefromorigin = df_distancefromorigin.reset_index(drop=True)
  df_distancefromorigin

  return df_distancefromorigin['ID'][0], df_distancefromorigin['distance'][0], df_distancefromorigin['distance_destination'][0], df_distancefromorigin['ev_charger'][0]

# Plot Route on Map
This is the main program function.  Given an Origin, Destination, Starting Range and Normal Range the function will iteratively call the get_distance_duration function to build a route from the origin to the destination.  The function ends when the final destination is within range of the final EV charger or a maximum number of stops have been reached.
The function will create a list of location (long and lat) with origin, destination and EV chargers.  Based on that route a call is made to Open Route Service to provide the directions using the EV Chargers as waypoints (stop points.)  Finally the returned route from Open Route Service is plotted on a Map object and the markers added to the Map.  The function returns the Map object

In [11]:
import openrouteservice
import folium
import polyline

def plot_route_on_map(loc_s, loc_e, op_startingrange, op_normalrange, max_stops):

  loc_origin = loc_s
  loc_destination = loc_e
  loc_evchargers = ls_ev_chargers_location[:3500] #List of EV Chargers Limit the amounr of calls to 3500 as that is the current max of Open Route Service Matrix Call

  df_distanceorigintodestination = get_distance_duration(loc_origin, [loc_destination], api_key_openrouteservice)

  current_range = op_startingrange
  initial_distance_nochargers = df_distanceorigintodestination[0]['distance']
  remaining_distance = df_distanceorigintodestination[0]['distance']
  loc_start = loc_origin
  i = 0
  df_routeinfo = []

  while remaining_distance > current_range:
    routeinfo = find_optimal_ev_charger(loc_start, loc_destination, loc_evchargers, current_range)
    loc_start = routeinfo[3]
    remaining_distance = routeinfo[2]
    current_range = op_normalrange
    df_routeinfo.append({
            'ID': routeinfo[0],
            'Distance_LastLeg': routeinfo[1],
            'Distance_Remaining': routeinfo[2],
            'Coordinates': routeinfo[3],
            'Coordinates reversed': [routeinfo[3][1]]+[routeinfo[3][0]] #Need to reverse Long and Lat for Plotting on Map
    })
    i = i + 1
    if i > max_stops:
      print("Something has gone wrong.  Cannot find a route")
      remaining_distance = 0

  coordinates = tuple([loc_origin])
  for x in df_routeinfo:
    coordinates = coordinates + tuple([x["Coordinates"]])

  coordinates = coordinates + tuple([loc_destination])
  df_routeinfo_V2 = pd.DataFrame(df_routeinfo)

  print(initial_distance_nochargers, "km Total Distance without charging")
  print(i, "EV Charger Stops Required")
  if i > 0:
    print( df_routeinfo_V2['Distance_LastLeg'].sum() + df_routeinfo_V2.iloc[-1]['Distance_Remaining'], "km Total Distance with charging")

  df_routeinfo_V2
  ##GET THE DIRECTIONS

  client = openrouteservice.Client(key=api_key_openrouteservice)

  # Get the directions
  directions = client.directions(coordinates, profile='driving-car', units='km')

  # Extract the route coordinates from the response
  route_geometry_polyline = directions['routes'][0]['geometry']

  # Decode the polyline to obtain the route coordinates
  route_coordinates = polyline.decode(route_geometry_polyline)

  loc_origin_marker = [loc_origin[1]] + [loc_origin[0]]
  loc_destination_marker = [loc_destination[1]] + [loc_destination[0]]

  if route_coordinates is not None:
    # Plotting the route on an actual map
    map_obj = folium.Map(width=900,height=600, location=[-25.5, 133.1], zoom_start=5)
    folium.PolyLine(locations=route_coordinates, color='blue').add_to(map_obj)
    folium.Marker(location=loc_origin_marker, popup='Origin').add_to(map_obj)
    for x in df_routeinfo:
      folium.Marker(location=x['Coordinates reversed'], popup=x['ID'], icon=folium.Icon(color='red')).add_to(map_obj)
    folium.Marker(location=loc_destination_marker, popup='Destination').add_to(map_obj)
    #display(map_obj)
  else:
    print("Failed to retrieve the joined route.")

  return map_obj

In [13]:
#@title Chameleon EV Charger Route Planning Service { run: "auto" }

# @markdown This is the range of the vehicle when charged to 100%
Vehicle_Max_Range = 400 #@param {type:"slider", min:0, max:1000, step:1}
# @markdown ---

# @markdown This is the current charge percent of the vehicle at the start of the journey.
Starting_Percent_Charge = 95 #@param {type:"slider", min:0, max:100, step:1}
# @markdown ---

# @markdown This is the maximum amount of charge that will be applied when stopping at an EV Charger.  When stopping and charging at charging stations the rate of charge (speed) is dependent on the current charge amount. Charging vehicles from 80% to 100% is typically much slower than the rate of charge from 10% to 80%. Therefore when travelling it is unlikely you will charge to 100% unless stopped overnight.
Max_Percent_Charge_at_Charger = 80 #@param {type:"slider", min:80, max:100, step:1}
# @markdown ---

# @markdown This is the amount of charge the user would like remaining when they reach a charging station. Think of this like the petrol light. The algorithm will limit the driving distance such that the vehicle charge percent will not go below the safety margin
Safety_Margin_Percent_Charge = 10 #@param {type:"slider", min:0, max:20, step:1}
# @markdown ---

# @markdown Location Type Selector.  Yes = Select from City List.  No = Select Long and Lat
Location_Type_Selector = True # @param {type:"boolean"}
# @markdown ---

# @markdown Select your origin and your destination from the list of available cities/towns
Location_Start_City = "Canberra" # @param ['Adelaide', 'Albany', 'Albury-Wodonga', 'Ballarat', 'Bathurst', 'Bendigo', 'Brisbane', 'Bunbury', 'Burnie', 'Cairns', 'Canberra', 'Central Coast', 'Coffs Harbour', 'Darwin', 'Devonport', 'Dubbo', 'Geelong', 'Geraldton', 'Gladstone', 'Gold Coast', 'Hervey Bay', 'Hobart', 'Kalgoorlie-Boulder', 'Launceston', 'Lismore', 'Mackay', 'Maitland', 'Melbourne', 'Mildura', 'Mount Gambier', 'Newcastle', 'Nowra', 'Orange', 'Perth', 'Port Macquarie', 'Rockhampton', 'Shepparton', 'Sunshine Coast', 'Sydney', 'Tamworth', 'Toowoomba', 'Townsville', 'Wagga Wagga', 'Warrnambool', 'Whyalla', 'Wollongong']
Location_End_City = "Melbourne" # @param ['Adelaide', 'Albany', 'Albury-Wodonga', 'Ballarat', 'Bathurst', 'Bendigo', 'Brisbane', 'Bunbury', 'Burnie', 'Cairns', 'Canberra', 'Central Coast', 'Coffs Harbour', 'Darwin', 'Devonport', 'Dubbo', 'Geelong', 'Geraldton', 'Gladstone', 'Gold Coast', 'Hervey Bay', 'Hobart', 'Kalgoorlie-Boulder', 'Launceston', 'Lismore', 'Mackay', 'Maitland', 'Melbourne', 'Mildura', 'Mount Gambier', 'Newcastle', 'Nowra', 'Orange', 'Perth', 'Port Macquarie', 'Rockhampton', 'Shepparton', 'Sunshine Coast', 'Sydney', 'Tamworth', 'Toowoomba', 'Townsville', 'Wagga Wagga', 'Warrnambool', 'Whyalla', 'Wollongong']
# @markdown ---

# @markdown If you have unselected the Location_Type_Selector then select you orign (long/lat) and destination (long/lat).  WARNING: There is no error checking on these coordinates!!!!!
Longtitude_Start = 145.1149 # @param {type:"number"}
Latitude_Start = -37.8475 # @param {type:"number"}
Longtitude_End = 141.4608 # @param {type:"number"}
Latitude_End = -31.9596 # @param {type:"number"}

user_maxrange = Vehicle_Max_Range
user_startingpercent = Starting_Percent_Charge / 100.0
user_safetymargin = Safety_Margin_Percent_Charge / 100.0
user_maxchargepercent = Max_Percent_Charge_at_Charger / 100.0

op_startingrange = (user_maxrange * user_startingpercent) - (user_maxrange * user_safetymargin)
op_normalrange = user_maxrange * user_maxchargepercent


print("Your Vehicle Max Range:", user_maxrange)
print("Your Starting Percent Charge:", user_startingpercent)
print("Your Max Percent Charge at Charger:", user_maxchargepercent)
print("Your Safety Margin Percent Charge:", user_safetymargin)
print("Operating Starting Range:", op_startingrange)
print("Operating Normal Range:", op_normalrange)

if Location_Type_Selector == True:
  Location_Start_Name = Location_Start_City
  Location_End_Name = Location_End_City
  Coordinates_Start = australian_cities_location[Location_Start_City]
  Coordinates_End = australian_cities_location[Location_End_City]
else:
  Location_Start_Name = "User Defined Start"
  Location_End_Name = "User Defined End"
  Coordinates_Start = [Longtitude_Start,Latitude_Start]
  Coordinates_End = [Longtitude_End,Latitude_End]

print("Location Start:", Location_Start_Name, Coordinates_Start)
print("Location End:", Location_End_Name, Coordinates_End)

import ipywidgets as widgets
from IPython.display import display
button = widgets.Button(description="Calc Route for ME!")
output = widgets.Output()

def on_button_clicked(b):
  # Display the message within the output widget.
  with output:
    if Location_Start_City == Location_End_City:
      print("C'mon you are better than that - Start and End are the same place!!!")
    else:
      print("Route Calculation in Progress!")
      display(plot_route_on_map(Coordinates_Start, Coordinates_End, op_startingrange, op_normalrange, const_maxstops))

button.on_click(on_button_clicked)
display(button, output)

Your Vehicle Max Range: 400
Your Starting Percent Charge: 0.95
Your Max Percent Charge at Charger: 0.8
Your Safety Margin Percent Charge: 0.1
Operating Starting Range: 340.0
Operating Normal Range: 320.0
Location Start: Canberra [149.13, -35.2809]
Location End: Melbourne [144.9631, -37.8136]


Button(description='Calc Route for ME!', style=ButtonStyle())

Output()